# Check if torch can detect GPU

In [5]:
import torch


class GPU:
    def __init__(self, device_id: int = 0):
        self.available = torch.cuda.is_available()
        self.device_id = device_id
        self.device = torch.device("cuda", device_id) if self.available else torch.device("cpu")
        self.name: str | None = None
        self.vram_total_gb: float | None = None
        self.vram_free_gb: float | None = None
        self.multiprocessors: int | None = None

        if self.available:
            props = torch.cuda.get_device_properties(device_id)
            self.name = props.name
            self.vram_total_gb = props.total_memory / (1024**3)
            self.multiprocessors = props.multi_processor_count
            try:
                free, _ = torch.cuda.mem_get_info(device_id)
                self.vram_free_gb = free / (1024**3)
            except Exception:
                pass

    def __str__(self) -> str:
        if not self.available:
            return "GPU(available=False, device=cpu)"

        lines = [
            f"GPU(device_id={self.device_id}, device={self.device})",
            f"  name:             {self.name}",
            f"  vram_total_gb:    {self.vram_total_gb:.2f}",
            f"  vram_free_gb:     {self.vram_free_gb:.2f}" if self.vram_free_gb is not None else "  vram_free_gb:     N/A",
            f"  multiprocessors:  {self.multiprocessors}",
        ]
        
        return "\n".join(lines)

    def __repr__(self) -> str:
        return self.__str__()


# Example usage
gpu = GPU()
print(gpu)

GPU(device_id=0, device=cuda:0)
  name:             AMD Radeon Graphics
  vram_total_gb:    15.98
  vram_free_gb:     15.59
  multiprocessors:  30


# Toy Project
A simple toy project to get the basic understanding of the Torch library.

## Neural Network

In [6]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs: int, num_outputs: int):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # Output layer
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logits = self.layers(x)
        return logits

In [7]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=50, num_outputs=3)
X = torch.randn((1, 50))

out = model(X)
print(out)

with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
    print(out)
    print(sum(out[0])) # class probabilities should sum to 1

tensor([[-0.0610,  0.1134, -0.0417]], grad_fn=<AddmmBackward0>)
tensor([[0.3115, 0.3709, 0.3176]])
tensor(1.)


## Data Loaders
![](../assets/image.png)

In [8]:
## Dummy data

X_train = torch.tensor([
[-1.2, 3.1],
[-0.9, 2.9],
[-0.5, 2.6],
[2.3, -1.1],
[2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
[-0.8, 2.8],
[2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [9]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return len(self.features)

In [10]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [11]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
)

for batch in train_loader:
    print(batch)

[tensor([[ 2.3000, -1.1000],
        [-1.2000,  3.1000]]), tensor([1, 0])]
[tensor([[-0.9000,  2.9000],
        [ 2.7000, -1.5000]]), tensor([0, 1])]
[tensor([[-0.5000,  2.6000]]), tensor([0])]


In [12]:
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels) # applies softmax internally

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
        f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
        f" | Train Loss: {loss.item():.2f}")

    model.eval()


Epoch: 001/003 | Batch 000/003 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/003 | Train Loss: 0.65
Epoch: 001/003 | Batch 002/003 | Train Loss: 0.42
Epoch: 002/003 | Batch 000/003 | Train Loss: 0.05
Epoch: 002/003 | Batch 001/003 | Train Loss: 0.13
Epoch: 002/003 | Batch 002/003 | Train Loss: 0.00
Epoch: 003/003 | Batch 000/003 | Train Loss: 0.01
Epoch: 003/003 | Batch 001/003 | Train Loss: 0.00
Epoch: 003/003 | Batch 002/003 | Train Loss: 0.02


In [13]:
model.eval()

with torch.no_grad():
    outputs = model(X_train)

print(outputs)

tensor([[ 2.9320, -4.2563],
        [ 2.6045, -3.8389],
        [ 2.1484, -3.2514],
        [-2.1461,  2.1496],
        [-2.5004,  2.5210]])


In [14]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[0.9992, 0.0008],
        [0.9984, 0.0016],
        [0.9955, 0.0045],
        [0.0134, 0.9866],
        [0.0066, 0.9934]])


In [15]:
predictions = torch.argmax(probas, dim=1)
print(predictions)


tensor([0, 0, 0, 1, 1])


In [16]:
print(predictions == y_train)

tensor([True, True, True, True, True])


In [17]:
def compute_accuracy(model: torch.nn.Module, dataloader: DataLoader):
    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = predictions == labels
        correct += torch.sum(compare)
        total_examples += len(compare)

    return correct / total_examples

print(compute_accuracy(model, test_loader))

tensor(1.)


In [18]:
tensor_1 = torch.tensor([1.0, 2.0, 3.0])
tensor_2 = torch.tensor([4.0, 5.0, 6.0])
print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


In [19]:
tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")
print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')
